Functions are defined with the def statement. The order and number of arguments must match those given in the function definition. If a mismatch exists, a TypeError exception is raised. When a function defines a parameter with a default value, that parameter and all the parameters that follow are optional. If values are not assigned to all the optional parame- ters in the function definition, a SyntaxError exception is raised.

Default parameter values are always set to the objects that were supplied as values when the function was defined. 

In [2]:
a = 10
def foo(x=a):
    return x

a = 5 # Reassign 'a'.
foo()

10

In addition, the use of mutable objects as default values may lead to unintended behavior

In [3]:
def foo(x, items=[]):
    items.append(x)
    return items

foo(1)
foo(2)
foo(3)

[1, 2, 3]

Notice how the default argument retains modifications made from previous invocations. To prevent this, it is better to use None and add a check as follows:

In [4]:
def foo(x, items=None):
    if items is None:
        items = []
    items.append(x)
    return items

A function can accept a variable number of parameters if an asterisk (*) is added to the last parameter name. In this case, all the remaining arguments are placed into the args variable as a tuple. To pass a tuple args to a function as if they were parameters, the *args syntax can be used in a function call.

If the last argument of a function definition begins with \**, all the additional keyword arguments (those that don’t match any of the other parameter names) are placed in a dictionary and passed to the function.This can be a useful way to write functions that accept a large number of potentially open-ended configuration options that would be too unwieldy to list as parameters.

In [ ]:
def fprintf(file, fmt, *args):
    file.write(fmt % args)

# Use fprintf. args gets (42,"hello world", 3.45) 
fprintf(out,"%d %s %f", 42, "hello world", 3.45)

Function arguments can also be supplied by explicitly naming each parameter and spec- ifying a value.These are known as keyword arguments.

def foo(w,x,y,z):
    statements

Keyword argument invocation 

foo(x=3, y=22, w='hello', z=[1,2])

With keyword arguments, the order of the parameters doesn’t matter. However, unless there are default values, you must explicitly name all of the required function parame- ters. If you omit any of the required parameters or if the name of a keyword doesn’t match any of the parameter names in the function definition, a TypeError exception is raised. Also, since any Python function can be called using the keyword calling style, it is generally a good idea to define functions with descriptive argument names. Positional arguments and keyword arguments can appear in the same function call, provided that all the positional arguments appear first, values are provided for all non- optional arguments, and no argument value is defined more than once. Here’s an example:

foo('hello', 3, z=[1,2], y=22)  
foo(3, 22, w='hello', z=[1,2]) # TypeError. Multiple values for w

In [7]:
# Accept variable number of positional or keyword arguments 
def spam(*args, **kwargs):
    pass
# args is a tuple of positional args # kwargs is dictionary of keyword args